# List of Exposures with Hologram at USDF


- work with Weakly_2022_39
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/09/19
- update : 2022/10/31



In [1]:
import sys
print(sys.executable)

/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-4.1.0/bin/python3


In [2]:
print(sys.path)

['/home/d/dagoret/notebooks/AuxTelComm/notebooks_usdf/butlertools', '/home/d/dagoret/repos/repos_w_2023_01/summit_utils/python', '/home/d/dagoret/repos/repos_w_2023_01/atmospec/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/spectractor/g90263d33d7/lib/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/obs_lsst/g0b8a85694d+5651ddfbf2/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/pipe_tasks/g369a80f31c+437b643543/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/shapelet/gd877ba84e5+60f67cdeb1/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/meas_modelfit/g9f262a3a84+375022659d/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/meas_extensions_shapeHSM/gc4a4d888fa+ef66f08a2a/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/scarlet_extensions/g9d18589735+cc492336a9/lib/python', '/opt/lsst/softwar

In [3]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2023_01/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-4.1.0/eups 	setup
summit_utils          LOCAL:/home/d/dagoret/repos/repos_w_2023_01/summit_utils 	setup


In [4]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

w_2022_39
   g0b29ad24fb+a10408d0bf 	current w_2022_39 setup


In [5]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits

In [6]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [7]:
import lsst.daf.butler as dafButler

In [8]:
#repo = "/sdf/group/rubin/repo/main"
repo="/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [9]:
collection='LATISS/raw/all'

## List of Exposures

In [10]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num', 'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle','science_program'])

In [11]:
for count, info in enumerate(registry.queryDimensionRecords('exposure',where= "instrument='LATISS'")):
    
    
    df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num, info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle, info.science_program]
    
    if count < 0:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
    

In [12]:
df_exposure

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle,science_program
0,2022083100004,AT_O_20220831_000004,20220831,4,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown
1,2022083100008,AT_O_20220831_000008,20220831,8,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown
2,2022083100007,AT_O_20220831_000007,20220831,7,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown
3,2022083100009,AT_O_20220831_000009,20220831,9,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown
4,2022083100012,AT_O_20220831_000012,20220831,12,dark,UNKNOWN,unknown~unknown,NaN,3.0,NaN,NaN,NaN,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30257,2023010500143,AT_O_20230105_000143,20230105,143,bias,UNKNOWN,empty~empty,None,0.0,None,None,None,unknown
30258,2023010500145,AT_O_20230105_000145,20230105,145,bias,UNKNOWN,empty~empty,None,0.0,None,None,None,unknown
30259,2023010500147,AT_O_20230105_000147,20230105,147,bias,UNKNOWN,empty~empty,None,0.0,None,None,None,unknown
30260,2023010500149,AT_O_20230105_000149,20230105,149,bias,UNKNOWN,empty~empty,None,0.0,None,None,None,unknown


## Selection of science

In [13]:
df_science = df_exposure[df_exposure.type == 'science']

In [14]:
df_science.sort_values(by="id",ascending=True,inplace=True)

/tmp/ipykernel_25013/3986782663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science.sort_values(by="id",ascending=True,inplace=True)


In [15]:
df_science.tail(60)

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle,science_program
29553,2022121200703,AT_O_20221212_000703,20221212,703,science,SPT-CL-J0559-5249_015,SDSSr_65mm~empty,25.510913,30.0,90.238724,-53.107959,0.001373,AUXTEL_DRP_IMAGING
29526,2022121200704,AT_O_20221212_000704,20221212,704,science,SPT-CL-J0559-5249_015,SDSSr_65mm~empty,25.555818,30.0,90.238797,-53.107979,0.001214,AUXTEL_DRP_IMAGING
29554,2022121200705,AT_O_20221212_000705,20221212,705,science,SPT-CL-J0559-5249_016,SDSSr_65mm~empty,25.569077,30.0,90.395536,-53.10799,0.001369,AUXTEL_DRP_IMAGING
29527,2022121200706,AT_O_20221212_000706,20221212,706,science,SPT-CL-J0559-5249_016,SDSSr_65mm~empty,25.612982,30.0,90.395587,-53.10798,0.001157,AUXTEL_DRP_IMAGING
29555,2022121200707,AT_O_20221212_000707,20221212,707,science,SPT-CL-J0559-5249_016,SDSSg_65mm~empty,25.675278,30.0,90.395493,-53.107979,0.001362,AUXTEL_DRP_IMAGING
29528,2022121200708,AT_O_20221212_000708,20221212,708,science,SPT-CL-J0559-5249_016,SDSSg_65mm~empty,25.719914,30.0,90.395547,-53.108018,0.001125,AUXTEL_DRP_IMAGING
29556,2022121200709,AT_O_20221212_000709,20221212,709,science,SPT-CL-J0559-5249_016,SDSSi_65mm~empty,25.783108,30.0,90.395479,-53.107946,0.001477,AUXTEL_DRP_IMAGING
29529,2022121200710,AT_O_20221212_000710,20221212,710,science,SPT-CL-J0559-5249_016,SDSSi_65mm~empty,25.828411,30.0,90.3956,-53.107993,0.001089,AUXTEL_DRP_IMAGING
29557,2022121200711,AT_O_20221212_000711,20221212,711,science,SPT-CL-J0559-5249_022,SDSSi_65mm~empty,25.92661,30.0,90.081702,-53.013987,0.001572,AUXTEL_DRP_IMAGING
29530,2022121200712,AT_O_20221212_000712,20221212,712,science,SPT-CL-J0559-5249_022,SDSSi_65mm~empty,25.973463,30.0,90.081744,-53.013983,0.001301,AUXTEL_DRP_IMAGING


In [16]:
df_science.day_obs.unique()

array([20220912, 20220913, 20220914, 20220927, 20220928, 20220929,
       20221011, 20221012, 20221013, 20221025, 20221026, 20221027,
       20221108, 20221109, 20221110, 20221122, 20221123, 20221124,
       20221207, 20221208, 20221209, 20221210, 20221212])

## Selection of date

2021-02-16  2021-02-18  2021-06-08  2021-06-10  2021-07-07  2021-09-07  2021-09-09  2021-10-06  2021-11-02  2021-11-04  2022-02-16  2022-03-15  2022-03-17  20220502  
2021-02-17  2021-03-11  2021-06-09  2021-07-06  2021-07-08  2021-09-08  2021-10-05  2021-10-07  2021-11-03  2022-02-15  2022-02-17  2022-03-16  2022-03-18  20220503 20220524 20220607 20220608 20220609

### Add those

20220504 - Not existing

In [17]:
#date_sel = 20220630
#date_sel = 20220630
#date_sel = 20221208
date_sel = 20221209

In [18]:
df_science_selected = df_science[df_science.day_obs == date_sel ]

In [19]:
len(df_science_selected)

36

# List of filters

In [20]:
list_of_filters = df_science_selected['filter'].unique()
list_of_filters

array(['empty~holo4_003'], dtype=object)

In [21]:
selected_filters = []
for filt in list_of_filters:
    #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
    flag_sel = (filt.find('holo4') != -1) 
    #flag_sel = (filt.find('ronchi170') != -1) 
    if flag_sel:
        selected_filters.append(filt) 
selected_filters = np.array(selected_filters)
selected_filters       

array(['empty~holo4_003'], dtype='<U15')

## Selection of filter

- But now we have a loop on filters

In [22]:
#idx_sel_filt = 0
#selected_filter = selected_filters[0]
#selected_filter

## Final selection of exposure

- loop on selected filters

In [23]:
pd.set_option('display.max_rows', None)

In [24]:
all_def_science_selected_final = []

for selected_filter in selected_filters:

    print(selected_filter)
    df_science_selected_final = df_science_selected[df_science_selected["filter"] ==  selected_filter]
    
    #print(df_science_selected_final)
    all_def_science_selected_final.append(df_science_selected_final.copy())
    
    df = df_science_selected_final
    df_sel = df[['day_obs', 'seq_num']]
    
    
    NN = len(df)
    
    if NN>0:
        # build the line
        
        df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
    
    
         # prepare output
        filename_out = "visitdispersers_"+ str(date_sel) + "_filt_" + selected_filter.replace('~','-') + ".list"
        print(filename_out)
        
        df_out = df_sel[["line"]]
        df_out.to_csv(filename_out, index=False, header=None)

empty~holo4_003
visitdispersers_20221209_filt_empty-holo4_003.list


/tmp/ipykernel_25013/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)


In [25]:
len(all_def_science_selected_final)

1

In [26]:
len(all_def_science_selected_final[0])

36

In [27]:
all_def_science_selected_final[0]

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle,science_program
28216,2022120900273,AT_O_20221209_000273,20221209,273,science,HD185975,empty~holo4_003,60.834493,30.0,307.10407,-87.466631,208.379128,spec
28186,2022120900274,AT_O_20221209_000274,20221209,274,science,HD185975,empty~holo4_003,60.841455,30.0,307.107478,-87.46677,208.375385,spec
28188,2022120900278,AT_O_20221209_000278,20221209,278,science,ETA1-DOR,empty~holo4_003,39.123792,30.0,91.530888,-66.044167,52.554009,spec
28219,2022120900279,AT_O_20221209_000279,20221209,279,science,ETA1-DOR,empty~holo4_003,39.080885,30.0,91.531072,-66.044136,52.553804,spec
28224,2022120900289,AT_O_20221209_000289,20221209,289,science,HD14943,empty~holo4_003,31.474966,30.0,35.725202,-51.092109,151.033755,spec
28194,2022120900290,AT_O_20221209_000290,20221209,290,science,HD14943,empty~holo4_003,31.573935,30.0,35.725186,-51.092089,151.033664,spec
28195,2022120900292,AT_O_20221209_000292,20221209,292,science,HD009051,empty~holo4_003,41.540885,30.0,22.191869,-24.340421,198.815389,spec
28226,2022120900293,AT_O_20221209_000293,20221209,293,science,HD009051,empty~holo4_003,41.695786,30.0,22.191842,-24.340412,198.811415,spec
28196,2022120900294,AT_O_20221209_000294,20221209,294,science,HD009051,empty~holo4_003,42.362035,30.0,22.191855,-24.340439,198.950969,spec
28227,2022120900295,AT_O_20221209_000295,20221209,295,science,HD009051,empty~holo4_003,42.516858,30.0,22.19184,-24.340419,198.950945,spec


In [28]:
all_def_science_selected_final[1]

IndexError: list index out of range

In [ ]:
all_def_science_selected_final[2]

In [ ]:
all_def_science_selected_final[3]

In [ ]:
all_def_science_selected_final[4]